In [1]:
import datetime as dt
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

pyo.init_notebook_mode(connected=True)
pd.options.plotting.backend = 'plotly'
dataset = pd.read_csv(r"./Datasets/DJI/DJI_MFDFA.csv")
df = dataset.reset_index(drop=True)
df

,Unnamed: 0.1,Unnamed: 0,Date,Open,High,Low,Close,QPLp1,QPLp2,QPLp3,...,QPLn13,QPLn14,QPLn15,QPLn16,QPLn17,QPLn18,QPLn19,QPLn20,QPLn21,Hurst
0,255,255,2011/1/6,11716.93,11736.74,11667.46,11697.31,11738.222156,11788.243849,11850.352381,...,10764.095394,10672.779158,10580.883918,10488.573450,10395.993221,10303.272878,10210.528274,10117.863137,10025.370464,0.509595
1,256,256,2011/1/7,11696.86,11726.94,11599.68,11674.76,11718.115685,11768.051695,11830.053841,...,10745.657510,10654.497690,10562.759858,10470.607509,10378.185861,10285.624339,10193.038599,10100.532188,10008.197947,0.515811
2,257,257,2011/1/10,11672.34,11677.33,11573.87,11637.45,11693.551127,11743.382457,11805.254628,...,10723.131506,10632.162783,10540.617260,10448.658089,10356.430183,10264.062697,10171.671043,10079.358553,9987.217870,0.523720
3,258,258,2011/1/11,11638.51,11704.12,11635.48,11671.88,11659.659651,11709.346554,11771.039401,...,10692.052601,10601.347534,10510.067338,10418.374692,10326.414091,10234.314314,10142.190439,10050.145499,9958.271868,0.520909
4,259,259,2011/1/12,11673.62,11782.23,11673.62,11755.44,11694.833453,11744.670247,11806.549204,...,10724.307415,10633.328716,10541.773154,10449.803899,10357.565879,10265.188264,10172.786478,10080.463865,9988.313078,0.521164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751,2006,2006,2017/12/20,24838.09,24852.44,24697.11,24726.65,24883.226098,24989.264395,25120.924933,...,22818.227144,22624.650764,22429.846985,22234.162987,22037.907135,21841.354265,21644.749966,21448.314123,21252.243879,0.542068
1752,2007,2007,2017/12/21,24778.26,24850.91,24766.27,24782.29,24823.287374,24929.070246,25060.413640,...,22763.262590,22570.152497,22375.817962,22180.605327,21984.822217,21788.742803,21592.612084,21396.649417,21201.051466,0.542207
1753,2008,2008,2017/12/22,24764.04,24784.15,24717.51,24754.06,24809.041533,24914.763698,25046.031715,...,22750.198977,22557.199708,22362.976700,22167.876095,21972.205343,21776.238458,21580.220296,21384.370090,21188.884391,0.547528
1754,2009,2009,2017/12/26,24715.84,24778.13,24708.42,24746.21,24760.753943,24866.270333,24997.282854,...,22705.918658,22513.295037,22319.450059,22124.729193,21929.439288,21733.853827,21538.217190,21342.748180,21147.642969,0.557199


In [2]:
maxLevel = 21                           
maxTP = 1                
maxTS = 4096             
nTP = 0                    
p3 = 1.0 / 3.0          

stime = 0
etime = 0
Gstime = 0
Getime = 0
tlapse = 0
Gtlapse = 0

day = 2261

DT_DT= np.zeros((day,1))

DT_OP= np.zeros((day,1))
DT_HI= np.zeros((day,1))
DT_LO= np.zeros((day,1))
DT_CL= np.zeros((day,1))
DT_VL= np.zeros((day,1))

cHurst= np.zeros((day,1))
oHurst= np.zeros((day,1))
hHurst= np.zeros((day,1))
lHurst= np.zeros((day,1))
DT_RT= np.zeros((day,1))

Q= np.zeros((100,1))
NQ= np.zeros((100,1))
r= np.zeros((100,1))


QFEL = np.zeros((21,1))       
QPR = np.zeros((21,1))       
NQPR = np.zeros((21,1))      
K = np.zeros((21,1))


ALL_Pos_QPL = np.zeros((day,21))
ALL_Neg_QPL = np.zeros((day,21))

print("Printout ALL K values K0 .. K20 for first 20 Levels")
for L in range(0,21):    
    K[L] = math.pow((1.1924 + (33.2383 * L) + (56.2169 * L * L)) / (1 + (43.6106 * L)), p3)
    print("Level ",L, " K",L, " = ", K[L])   

Printout ALL K values K0 .. K20 for first 20 Levels
Level  0  K 0  =  [1.06041043]
Level  1  K 1  =  [1.2665998]
Level  2  K 2  =  [1.4911995]
Level  3  K 3  =  [1.66349934]
Level  4  K 4  =  [1.80609901]
Level  5  K 5  =  [1.92919156]
Level  6  K 6  =  [2.03832293]
Level  7  K 7  =  [2.13688129]
Level  8  K 8  =  [2.22710524]
Level  9  K 9  =  [2.31055991]
Level  10  K 10  =  [2.38838747]
Level  11  K 11  =  [2.46144992]
Level  12  K 12  =  [2.53041565]
Level  13  K 13  =  [2.59581465]
Level  14  K 14  =  [2.65807505]
Level  15  K 15  =  [2.71754825]
Level  16  K 16  =  [2.77452666]
Level  17  K 17  =  [2.82925649]
Level  18  K 18  =  [2.88194724]
Level  19  K 19  =  [2.93277882]
Level  20  K 20  =  [2.98190702]


In [3]:
DT_DT = df['Date']     
DT_OP = df['Open']
DT_HI = df['High']
DT_LO = df['Low']
DT_CL = df['Close']
hurst = df['Hurst']
TSsize = df.shape[0]

TSsize

1756

In [4]:
# For Hurst
maxRno = TSsize - 2

for d in range (0,TSsize - 2):
    if (DT_CL[d + 1] > 0):
        DT_RT[d] = DT_CL[d] / DT_CL[d + 1]          
    else:
        DT_RT[d] = 1
        
mu = 0
for d in range(0,maxRno):
    mu = mu + DT_RT[d]
mu = mu / maxRno

sigma = 0
for d in range(0,maxRno):
    sigma = sigma + (DT_RT[d] - mu) * (DT_RT[d] - mu)
    
sigma = math.sqrt((sigma/ maxRno))

dr = 3 * sigma/ 50

auxR = 0
tQno = 0
num = 0

for nR in range (0,maxRno):    
    bFound = False
    nQ = 0
    # Get the start position of the wave function.
    auxR = 1 - (dr * 50)
    # Get the total number of the returns in each range of each segment of the wave function.
    while((bFound == False) and (nQ < 100)):
        
        if ((DT_RT[nR] > auxR) and (DT_RT[nR] <= (auxR + dr))):
            
            Q[nQ] = Q[nQ] + 1
            tQno= tQno + 1
            bFound = True
        else:
            num = num+1
            nQ = nQ + 1
            auxR = auxR + dr

auxR = 1 - (dr * 50)
for nQ in range(0,100):
    r[nQ] = auxR
    NQ[nQ] = Q[nQ] / tQno        
    auxR = auxR + dr

maxQ = 0
maxQno = 0

for nQ in range(0,100):     
    if (NQ[nQ] > maxQ):
        maxQ = NQ[nQ]
        maxQno = nQ

for i in range(0,day):     
    r0 = r[maxQno] - (dr / 2)
    r1 = r0 + dr
    rn1 = r0 - dr
    
for i in range(0,day):  
    Lup = (pow(rn1, 2) * NQ[maxQno - 1]) - (pow(r1, 2) * NQ[maxQno + 1])
    Ldw = (pow(rn1, 4) * NQ[maxQno - 1]) - (pow(r1, 4) * NQ[maxQno + 1])

L = abs(Lup / Ldw)

for eL in range(0,21):
    p = -1 * pow((2 * eL + 1), 2)
    q = -1 * L * pow((2 * eL + 1), 3) * pow(K[eL], 3)
    u = pow((-0.5 * q + np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    v = pow((-0.5 * q - np.sqrt(((q * q / 4.0) + (p * p * p / 27.0)))), p3)
    QFEL[eL] = u + v

for eL in range(0,21):
# Compute the QPR.  
    QPR[eL] = QFEL[eL] / QFEL[0]
#Compute the NQPR.
    NQPR[eL] = 1 + 0.21 * sigma * QPR[eL]

cQHLdata = pd.DataFrame(columns=["Date", "Open", "High", "Low", "Close", "cQHLp1", "cQHLp2", "cQHLp3", "cQHLp4", "cQHLp5", "cQHLp6", "cQHLp7", "cQHLp8",
      "cQHLp9", "cQHLp10", "cQHLp11", "cQHLp12", "cQHLp13", "cQHLp14", "cQHLp15", "cQHLp16", "cQHLp17", "cQHLp18", "cQHLp19", "cQHLp20", "cQHLp21", "cQHLn1", "cQHLn2",
      "cQHLn3", "cQHLn4", "cQHLn5", "cQHLn6", "cQHLn7", "cQHLn8", "cQHLn9", "cQHLn10", "cQHLn11", "cQHLn12", "cQHLn13", "cQHLn14", "cQHLn15", "cQHLn16",
      "cQHLn17", "cQHLn18", "cQHLn19", "cQHLn20", "cQHLn21"])

for d in range(0,TSsize):
    for eL in range(0,21):
        ALL_Pos_QPL[d][eL] = hurst[d] * NQPR[eL]
        ALL_Neg_QPL[d][eL] = hurst[d] / NQPR[eL]

cQHLdata.Date= DT_DT
cQHLdata.Open= DT_OP
cQHLdata.High= DT_HI
cQHLdata.Low = DT_LO
cQHLdata.Close = DT_CL
cqhlv = cQHLdata.values

for d in range(0,TSsize - 2):
    for eL in range(0,21):
        cqhlv[d][eL+5] = ALL_Pos_QPL[d][eL]
        cqhlv[d][eL+26] = ALL_Neg_QPL[d][eL]

cQHLdata = pd.DataFrame(cqhlv,columns=["Date", "Open", "High", "Low", "Close", "cQHLp1", "cQHLp2", "cQHLp3", "cQHLp4", "cQHLp5", "cQHLp6", "cQHLp7", "cQHLp8",
      "cQHLp9", "cQHLp10", "cQHLp11", "cQHLp12", "cQHLp13", "cQHLp14", "cQHLp15", "cQHLp16", "cQHLp17", "cQHLp18", "cQHLp19", "cQHLp20", "cQHLp21", "cQHLn1", "cQHLn2",
      "cQHLn3", "cQHLn4", "cQHLn5", "cQHLn6", "cQHLn7", "cQHLn8", "cQHLn9", "cQHLn10", "cQHLn11", "cQHLn12", "cQHLn13", "cQHLn14", "cQHLn15", "cQHLn16",
      "cQHLn17", "cQHLn18", "cQHLn19", "cQHLn20", "cQHLn21"])

dfc = pd.DataFrame(cQHLdata)
dfc.to_csv("./Datasets/DJI/DJI_MFDFA+qhl.csv",index = True)